In [22]:
import pandas as pd
db = pd.read_pickle('static/lexicon.pkl')

In [6]:
import json
with open('wiki_data_trans.json', 'r') as f:
    wiki_data = json.load(f)

In [8]:
ph_dict = {}
for k in wiki_data:
    if 'pronunciation' in wiki_data[k]:
        pronunciation = wiki_data[k]['pronunciation']
        ph_dict[k]= {
        'pronunciation':pronunciation
        }

ph_dict

{'assistance': {'pronunciation': {'US': '/əˈsɪstəns/'}},
 'unfortunately': {'pronunciation': {'UK': '/ʌnˈfɔːtjʊnətli/, /ʌnˈfɔːt͡ʃənətli/',
   'US': '/ʌnˈfɔɹt͡ʃənɪtli/'}},
 'alright': {'pronunciation': {'IPA': '/ˌɔːlˈɹaɪt/'}},
 'intelligence': {'pronunciation': {'IPA': '/ɪnˈtɛl.ɪ.d͡ʒəns/'}},
 'solve': {'pronunciation': {'UK': '/sɒlv/', 'US': '/sɑlv/, /sɔlv/'}},
 'tall': {'pronunciation': {'UK': '/tɔːl/', 'US': '/tɔl/'}},
 'fault': {'pronunciation': {'UK': '/fɔːlt/, /fɒlt/', 'US': '/fɔlt/'}},
 'forgive': {'pronunciation': {'US': '/fə(ɹ)ˈɡɪv/, /fɔː(ɹ)ˈɡɪv/'}},
 'he': {'pronunciation': {'IPA': '/ˈhiː/, (unstressed) IPA: /hi/, /i/'}},
 'opposite': {'pronunciation': {'UK': '/ˈɒpəzɪt/, /ˈɒpəsɪt/',
   'US': '/ˈɑp(ə)sɪt/, /ˈɑpəzɪt/'}},
 'slowly': {'pronunciation': {'US': '/ˈsləʊli/'}},
 'important': {'pronunciation': {'UK': '/ɪmˈpɔːtənt/', 'US': '/ɪmˈpɔ˞ʔənt/'}},
 'everybody': {'pronunciation': {'IPA': '/ˈɛvribɒdi/'}},
 'sheep': {'pronunciation': {'IPA': '/ʃiːp/'}},
 'pencil': {'pronunciation':

In [4]:
import json
with open('static/phAids.json', 'r') as f:
    aids = json.load(f)


In [ ]:

for k,v in aids.items():
    if k in ph_dict:
        ph_dict[k]['aid']=v
    else:
        ph_dict[k] = {
           'aid':v 
        }

In [12]:

ph_dict['make']

{'pronunciation': {'IPA': '/meɪk/, [meɪkʲ]'}, 'aid': ['    eɪ']}

In [9]:

for k,v in aids.items():
    entry = db[db['word'] == k]
    for i, row in entry.iterrows():
        phonetics = db.at[i,('phonetics')]
        if isinstance(phonetics, dict):
            phonetics['aid'] = v
        else:
            phonetics = {'aid': v}
        db.at[i,('phonetics')] = phonetics

In [11]:
db.to_pickle('lexicon.pkl')

In [27]:
db[db['word']=='always']['phonetics'].to_dict()

{421: {'pronunciation': {'UK': '/ˈɔː(l).weɪz/, /ˈɔːl.wɪz/',
   'US': '/ˈɔl.weɪz/, /ˈɔl.wiz/, /ˈɔl.wəz/'},
  'aid': ["'", 'ɔ      eɪ z']}}

In [28]:
db[db['word']=='determined']['phonetics'].to_dict()

{1997: {'pronunciation': {'UK': '/dɪˈtɜːmɪnd/', 'US': '/dɪˈtɝmɪnd/'},
  'aid': ['   ɪ ˈ ɝ     ɪ   ', 'x']}}